In [172]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [173]:
#ดูdata ว่าเป็นยังไง
df =pd.read_csv('message.csv')
df.head(2)

,Account,Message,Direct URL,Post URL,Comment URL,Reply comment URL,Source,Post time,Engagement,Main keyword,Sub keyword,Follower count,Sentiment,Category,Track post,Track account,Note
0,bangkokbiznews,กรมธุรกิจฯ ถกผู้ค้าน้ำมัน เร่งแผนกระตุ้นยอดใช้...,https://www.bangkokbiznews.com/news/detail/864...,https://www.bangkokbiznews.com/news/detail/864...,NaN,NaN,news,2020-02-04 00:00:00,0,pm 2.5,-,-,Neutral,-,NaN,NaN,-
1,bangkok-today,ศิลปิน “ยุ้งข้าวเรคคอร์ด” ตระหนักภาวะฝุ่นเป็นพ...,https://bangkok-today.com/web/ศิลปิน-ยุ้งข้าวเ...,https://bangkok-today.com/web/ศิลปิน-ยุ้งข้าวเ...,NaN,NaN,news,2020-02-04 00:00:00,0,"pm 2.5,ฝุ่นละอองขนาดเล็ก",-,-,Neutral,-,NaN,NaN,-


In [174]:
#dropสิ่งที่ไม่จำเป็นออกจากdataset
df.drop(['Account','Direct URL','Post URL','Comment URL','Reply comment URL','Engagement','Sub keyword','Follower count','Category','Track post','Track account','Note'],axis=1,inplace=True)

In [175]:
#เช็คว่าข้อมมูลได้มาจากไหนบ้าง ซึ่งคุณภาพจะแตกต่างออกไป
df['Source'].value_counts()

facebook     27269
instagram    12733
twitter       7234
news          3130
youtube       1252
forum         1123
blog           209
Name: Source, dtype: int64

In [176]:
#drop ข้อมูลที่มาจากแหล่งข่าวทางการ เพราะหลังจากที่ส่องดู มันมีแต่พาดหัวข่าว
df_use = df.loc[~df['Source'].isin(['news','forum','blog'])]

In [177]:
#ส่องdataดู
df_use.head()

,Message,Source,Post time,Main keyword,Sentiment
54,จริงที่สุดเลยค่ะ เคยเช็คที่Air4Thai ผลขึ้นมามี...,facebook,2020-02-04 00:02:52,pm2.5,Positive
55,Pm 2.5 คือควันรถยนต์ ก่อสร้างมันไม่ใช่สาเหตุ,facebook,2020-02-04 00:02:57,pm 2.5,Neutral
56,@hamham310715 ดูแลตัวเองด้วยนะครับฝุ่นpm2.5เยอ...,twitter,2020-02-04 00:04:06,pm2.5,Neutral
57,คิดกันบ้างไหมว่าทำไม PM2.5ถึงเกิดหนักเฉพาะช่วง...,facebook,2020-02-04 00:04:31,pm2.5,Neutral
58,บิ๊กตู่ ฮึ่ม จัดการคนปล่อยข่าวปลอม . วันที่ 3 ...,facebook,2020-02-04 00:04:56,pm2.5,Negative


<h1>ตั้งข้อสังเกตุได้ดังนี้</h1>
ข้อมูลที่ได้มาแล้วส่อแววจะมีปัญหา ก็คือ
1.# คือข้อมูลที่ติดแทก จะเป็นพวกขายของออนไลน์ใต้คอมเมนท์ 
2. @ คือ การรีไพลทวีตแล้วแทกเจ้าของทวีต ก่อนตอบกลับ 
ซึ่งประโยคที่ติดคำพวกนี้มันไม่จำเป็น เราจึงตัดมันออก


In [178]:
#สร้างฟังชั่นในการตัดประโยคที่ติดกับสัญลักษณืที่มีปัญหา
def cut_sentence(sent):
    return sent.split()
def del_words_insent(sent):
    arr_sent = cut_sentence(sent)
    for i in range(len(arr_sent)):      
        for word in arr_sent:
            count = 0
            checkdata = True
            for char in word:
                if char == '@':    
                    count += 1
                if char == '#':
                    count += 1
            if count >0:
                arr_sent.remove(word)

                
    return ''.join(arr_sent)

In [185]:
#ตัวอย่าง
del_words_insent('@mkmoo ถ้าทุกคนลำบาก หาที่ป้องกันฝุ่นpm2.5 เหลือไม่เยอะแล้วนะจ้า 79บาท พร้อมส่ง #เจลล้างมือ#เจลล้างมือพกพา#เจลล้างมืออนามัย#เจลล้างมือแบบไม่ต้องล้างออก#ไวรัสโคโรน่า#เจลล้างมือฆ่าเชื้อ#แอลกฮอล์ล้างมือ#ผ้าปิดปาสีดำ#หน้ากากอนามัย#มาร์กสีดำ#สินค้าขายดีของญี่ปุ่น#pitta mask#อ่อนโยนต่อใบหน้า#ป้องกันมลพิษ#ป้องกันUV#ป้องกันเชื้อโรค#ป้องกันฝุ่นPM2.5')


'ถ้าทุกคนลำบากหาที่ป้องกันฝุ่นpm2.5เหลือไม่เยอะแล้วนะจ้า79บาทพร้อมส่ง'

In [179]:
df_use

,Message,Source,Post time,Main keyword,Sentiment
54,จริงที่สุดเลยค่ะ เคยเช็คที่Air4Thai ผลขึ้นมามี...,facebook,2020-02-04 00:02:52,pm2.5,Positive
55,Pm 2.5 คือควันรถยนต์ ก่อสร้างมันไม่ใช่สาเหตุ,facebook,2020-02-04 00:02:57,pm 2.5,Neutral
56,@hamham310715 ดูแลตัวเองด้วยนะครับฝุ่นpm2.5เยอ...,twitter,2020-02-04 00:04:06,pm2.5,Neutral
57,คิดกันบ้างไหมว่าทำไม PM2.5ถึงเกิดหนักเฉพาะช่วง...,facebook,2020-02-04 00:04:31,pm2.5,Neutral
58,บิ๊กตู่ ฮึ่ม จัดการคนปล่อยข่าวปลอม . วันที่ 3 ...,facebook,2020-02-04 00:04:56,pm2.5,Negative
...,...,...,...,...,...
52945,ถ้าจำต้องนั่งตากฝุ่น PM2.5 บนดาดฟ้า..กับใครสั...,facebook,2020-03-04 11:00:43,pm2.5,Neutral
52946,หลายคนอาจสงสัยไทยมีผู้ติดเชื้อก่อนหลายประเทศแต...,facebook,2020-03-04 11:00:59,pm2.5,Positive
52947,ถ้าจำต้องนั่งตากฝุ่น PM2.5 บนดาดฟ้า..กับใครสั...,twitter,2020-03-04 11:02:10,pm2.5,Neutral
52948,ไม่พบโรงงานปล่อย pm2.5 ไม่พบรถเมล์ปล่อยควันดำ ...,facebook,2020-03-04 11:04:17,pm2.5,Neutral


In [180]:
#นำประโยคออกมาเป็นarray
df_use_arr = df_use['Message'].values

In [181]:
#แก้ไขประโยคเหล่านั้น โดยตัดคำที่มีปัญหาออกไป อาจจะรันนานหน่อย5555 ประมาณ2นาทีกว่า
new_df_use_arr = []
for i in df_use_arr:
    clean_sent = del_words_insent(str(i))
    new_df_use_arr.append(clean_sent)

In [182]:
#นำประโยคใหม่ใส่เข้าไปในของเก่า
df_use.insert(
    0,
    'Message_clean@',
    new_df_use_arr
)

In [183]:
np.array(new_df_use_arr).shape

(48488,)

In [184]:
df_use

,Message_clean@,Message,Source,Post time,Main keyword,Sentiment
54,จริงที่สุดเลยค่ะเคยเช็คที่Air4Thaiผลขึ้นมามีแต...,จริงที่สุดเลยค่ะ เคยเช็คที่Air4Thai ผลขึ้นมามี...,facebook,2020-02-04 00:02:52,pm2.5,Positive
55,Pm2.5คือควันรถยนต์ก่อสร้างมันไม่ใช่สาเหตุ,Pm 2.5 คือควันรถยนต์ ก่อสร้างมันไม่ใช่สาเหตุ,facebook,2020-02-04 00:02:57,pm 2.5,Neutral
56,ดูแลตัวเองด้วยนะครับฝุ่นpm2.5เยอะน้ำมูกไหลเลย,@hamham310715 ดูแลตัวเองด้วยนะครับฝุ่นpm2.5เยอ...,twitter,2020-02-04 00:04:06,pm2.5,Neutral
57,คิดกันบ้างไหมว่าทำไมPM2.5ถึงเกิดหนักเฉพาะช่วงต...,คิดกันบ้างไหมว่าทำไม PM2.5ถึงเกิดหนักเฉพาะช่วง...,facebook,2020-02-04 00:04:31,pm2.5,Neutral
58,บิ๊กตู่ฮึ่มจัดการคนปล่อยข่าวปลอม.วันที่3ม.ค.63...,บิ๊กตู่ ฮึ่ม จัดการคนปล่อยข่าวปลอม . วันที่ 3 ...,facebook,2020-02-04 00:04:56,pm2.5,Negative
...,...,...,...,...,...,...
52945,ถ้าจำต้องนั่งตากฝุ่นPM2.5บนดาดฟ้า..กับใครสักคน...,ถ้าจำต้องนั่งตากฝุ่น PM2.5 บนดาดฟ้า..กับใครสั...,facebook,2020-03-04 11:00:43,pm2.5,Neutral
52946,หลายคนอาจสงสัยไทยมีผู้ติดเชื้อก่อนหลายประเทศแต...,หลายคนอาจสงสัยไทยมีผู้ติดเชื้อก่อนหลายประเทศแต...,facebook,2020-03-04 11:00:59,pm2.5,Positive
52947,ถ้าจำต้องนั่งตากฝุ่นPM2.5บนดาดฟ้า..กับใครสักคน...,ถ้าจำต้องนั่งตากฝุ่น PM2.5 บนดาดฟ้า..กับใครสั...,twitter,2020-03-04 11:02:10,pm2.5,Neutral
52948,ไม่พบโรงงานปล่อยpm2.5ไม่พบรถเมล์ปล่อยควันดำไม่...,ไม่พบโรงงานปล่อย pm2.5 ไม่พบรถเมล์ปล่อยควันดำ ...,facebook,2020-03-04 11:04:17,pm2.5,Neutral
